In [1]:
from Preprocessing_M import DataExtractor, DataPreprocessing

de = DataExtractor("D:/hka-aqm-m", "C:/Users/danie/building_M_data")

df = de.create_df()

No .dat files found in C:/Users/danie/building_M_data. 
 Trying to extract files from the original directory D:/hka-aqm-m
Read data successfully.
Data contains 979166 data points and 18 columns.


In [15]:
dp = DataPreprocessing()

df_preprocessed = dp.preprocess_df(df, rolling_window = "7d", sample_time = "1d")

In [23]:
df_preprocessed

,date_time,tmp,hum,CO2,VOC,vis,IR,BLE,rssi,snr,...,tmp_diff_per_sec,hum_diff_per_sec,IR_diff_per_sec,vis_diff_per_sec,year,month,dayofweek,hour,color,VOC_CO2_ratio
0,2022-06-01,25.259375,31.653750,573.500000,450.062500,187.375000,86.687500,22.437500,-102.937500,6.037500,...,0.0,0.0,0.0,0.0,1,6,2,0,green,0.7848
1,2022-06-02,25.167642,33.097784,538.500000,450.531250,124.801136,49.480114,11.900568,-80.446023,6.850568,...,0.0,0.0,0.0,0.0,1,6,3,0,green,0.8366
2,2022-06-03,25.167642,33.097784,538.500000,450.531250,124.801136,49.480114,11.900568,-80.446023,6.850568,...,0.0,0.0,0.0,0.0,1,6,4,0,green,0.8366
3,2022-06-04,25.167642,33.097784,538.500000,450.531250,124.801136,49.480114,11.900568,-80.446023,6.850568,...,0.0,0.0,0.0,0.0,1,6,5,0,green,0.8366
4,2022-06-05,25.167642,33.097784,538.500000,450.531250,124.801136,49.480114,11.900568,-80.446023,6.850568,...,0.0,0.0,0.0,0.0,1,6,6,0,green,0.8366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7096,2023-09-26,22.853257,57.471834,504.646595,1856.179747,18.128860,5.515351,2.291867,-133.077878,-14.270139,...,0.0,0.0,0.0,0.0,2,9,1,0,green,3.6782
7097,2023-09-26,25.449370,47.429120,493.825879,784.360855,164.281825,42.421154,2.456934,-113.384334,0.220531,...,0.0,0.0,0.0,0.0,2,9,1,0,green,1.5883
7098,2023-09-26,22.511175,56.991668,507.738404,889.645760,79.485266,21.971884,3.436426,-121.260920,-4.887202,...,0.0,0.0,0.0,0.0,2,9,1,0,green,1.7522
7099,2023-09-26,24.580378,47.155862,472.886374,1076.529261,215.996879,78.389175,4.638188,-131.910133,-15.125274,...,0.0,0.0,0.0,0.0,2,9,1,0,green,2.2765


In [3]:
df_preprocessed.to_parquet("C:/Users/danie/hka_M_preprocessed.parquet")

In [4]:
import pandas as pd
df_preprocessed = pd.read_parquet("C:/Users/danie/hka_M_preprocessed.parquet")

In [5]:
from Feature_Engineering import *

In [6]:
fe = FeatureEngineering(df_preprocessed[df_preprocessed.room_number == "m001"], label = "tmp", categorical_features = ["room_number", "dayofweek", "month", "color"])

"['building_name'] not found in axis"


In [7]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(fe.X_train, fe.y_train)

LinearRegression()

In [8]:
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score

pred_test = lr.predict(fe.X_test)

pred_train = lr.predict(fe.X_train)

print(f"Training score: {r2_score(y_true = fe.y_train, y_pred = pred_train)}")

print(f"Training score: {r2_score(y_true = fe.y_test, y_pred = pred_test)}")

Training score: 0.8388153039076675
Training score: -0.5114510738241225


In [9]:
import plotly.express as px

px.scatter(x = fe.y_train.index, y = [fe.y_train, pred_train])

In [10]:
import plotly.express as px

px.scatter(x = fe.y_test.index, y = [fe.y_test, pred_test])

In [13]:
import tensorflow as tf
from keras.layers import LSTM, Dense, GRU, Dropout, Bidirectional